# QLoRA

Revisión informal de "_QLoRA: Efficient Finetuning of Quantized LLMs_".

Técnica publicada en mayo de 2023.

- [https://arxiv.org/pdf/2305.14314](https://arxiv.org/pdf/2305.14314)

Propone aplicar _LoRA_ sobre las matrices de pesos preentrenadas cuantizadas a 4 bits.

## 1. Arquitectura

_LoRA_ introduce matrices de bajo rango para añadir nuevo conocimiento a un modelo preentrenado mediante _fine-tuning_.

```
Weights x Input + (Weights' x Input) ─> Output
```

_QLoRA_ propone realizar el _fine-tuning_ con los pesos preentrenados del modelo cuantizados a 4 bits, reduciendo el consumo de memoria y mejorando el rendimiento.

```
Quantize(Weights) x Input + (Weights' x Input) ─> Output
```

## 2. Análisis

Supone que las matrices de pesos preentrenadas siguen distribuciones normales, propone un nuevo tipo de dato de 4 bits óptimo para cuantizar dicho tipo de distribuciones, y cuantiza las matrices con el nuevo tipo de dato para el _fine-tuning_.

### 2.1. 4-bit NormalFloat Quantization 

La cuantificación es el proceso de transformar valores continuos en valores discretos. Pero también es el proceso de discretizar valores representados con un tipo de dato determinado en otro utilizando menos bits.

_QLoRA_ introduce un nuevo tipo de datos llamado _4-bit NormalFloat (NF4)_ para la cuantización, que minimiza la pérdida de información para valores que siguen una distribución normal estándar $\mathcal{N}(0, 1)$ con media cero y desviación típica uno.

Describe el proceso de construcción de un nuevo tipo genérico de $k$-bits como la obtención de $2^k + 1$ cuantiles sobre una distribución normal estándar. Siendo los cuantiles puntos que dividen una distribución en partes iguales.

Pero detalla que para _NF4_ se calculan $2^{k-1}$ cuantiles para el intervalo negativo, $2^{k-1} + 1$ cuantiles para el positivo, se elimina una de las representaciones de $0$, y los cuantiles así obtenidos se normalizan al rango $[-1, 1]$.

Los 16 valores concretos calculados para el tipo _NF4_ se listan en un apéndice del _paper_.

```
[
 -1.0,
 -0.6961928009986877, -0.5250730514526367, -0.39491748809814453,
 -0.28444138169288635, -0.18477343022823334, -0.09105003625154495,
 0.0,
 0.07958029955625534, 0.16093020141124725, 0.24611230194568634,
 0.33791524171829224, 0.44070982933044434, 0.5626170039176941,
 0.7229568362236023,
 1.0
]
```

### 2.2. Quantization

Antes del proceso de _fine-tuning_, las matrices de pesos preentrenadas se normalizan, diviéndolas por su mayor valor absoluto, para llevarlas al mismo rango $[-1, 1]$ del tipo de dato _NF4_.

Dado que los pesos de las redes neuronales preentrenadas suelen tener una distribución normal centrada en cero (como _NF4_) y desviación típica $\sigma$. Esto tiene el efecto de reescalar la desviación típica $\sigma$ para que coincida con la desviación típica del tipo de dato _NF4_.

El problema de este enfoque es que un valor de la matriz muy alejado del resto (_outlier_) puede hacer que los datos se compriman perdiendo demasiada información. Por lo que las matrices, expresadas habitualmente como tensores $W \in \mathbb{R}^{b \times h}$, se procesan en $n$ bloques de tamaño $B$.

- $ n = \dfrac{b \times h}{B} $

Los bloques de valores, normalizados al intervalo $[-1, 1]$, se cuantizan mapeando cada uno de ellos a su valor más cercano dentro del conjunto de 16 valores predefinidos de _NF4_.

- $ W_i^{\text{NF4}} = \operatorname{quantize}\left(\dfrac{1}{c_2^i} W_i\right) $

Siendo $c_2^i$ el mayor valor absoluto de un bloque $i$, denominado _constante de cuantización_ (_quantization constant_) de dicho bloque.

- $ c_2^i  = \operatorname{absmax}(W_i) $

### 2.3. Double Quantization

Como resultado de la cuantización se obtienen los pesos $W^{\text{NF4}}$ cuantizados al tipo de dato _NF4_, y las constantes de cuantización de todos los bloques $c_2^{\text{FP32}}$ como números en punto flotante de 32 bits (_FP32_).

Pero para lograr una cuantización precisa de 4 bits se requieren bloques de tamaño pequeño. Y para cada bloque se necesita una constante de cuantización de 32 bits, lo que requiere mucha memoria para modelos grandes.

_QLoRA_ propone realizar una segunda cuantización, pero esta vez de las constantes.

Se calculan las constantes $c_2^{\text{FP32}}$ en bloques de tamaño 64, se calcula la media de todas las constantes, y se resta la media a cada constante para centrarlas alrededor de cero.

Las constantes centradas alrededor de cero se agrupan en bloques de tamaño 256, y se cuantizan a números de punto flotante de 8 bits, para obtener las nuevas constantes $c_2^{\text{FP8}}$.

Como resultado de la segunda cuantización, se obtiene un segundo nivel de constantes de cuantización $c_1^{\text{FP32}}$.

### 2.4. Entrenamiento

Durante el entrenamiento la matriz de pesos preentrenados cuantizada se congela y se aprenden dos nuevas matrices siguiendo la técnica original propuesta por _LoRA_.

- [https://github.com/jcmellado/sapiens/blob/main/notebooks/lora/lora.ipynb](https://github.com/jcmellado/sapiens/blob/main/notebooks/lora/lora.ipynb)

En el _paper_, tanto las matrices a entrenar, como todos los cálculos a realizar, se representan en el tipo de dato _BF16_. Formato de punto flotante en 16 bits, con un bit de signo, exponente de 8 bits, y una mantisa de 7 bits.

La salida de una capa lineal utilizando _QLoRA_ puede representarse mediante la siguiente expresión:

- $ Y^{\text{BF16}} = X^{\text{BF16}} \operatorname{doubleDequant}(c_1^{\text{FP32}}, c_2^{\text{FP8}}, W^{\text{NF4}}) + X^{\text{BF16}} L_1^{\text{BF16}} L_2^{\text{BF16}} $

Siendo:

- $Y^{\text{BF16}}$: la salida de la capa.

- $X^{\text{BF16}}$: la entrada de la capa.

- $\operatorname{doubleDequant}$: la función que reconstruye los valores de los pesos en precisión _BF16_ a partir de los cuantizados.

- $c_1^{\text{FP32}}$: las constantes de cuantización de segundo nivel.

- $c_2^{\text{FP8}}$: las constantes de cuantización de primer nivel.

- $W^{\text{NF4}}$: la matriz de pesos preentrenados de la capa cuantizada.

- $L_1^{\text{BF16}}$: la primera matriz de la composición de bajo rango.

- $L_2^{\text{BF16}}$: la segunda matriz de la composición de bajo rango.

La función $\operatorname{doubleDequant}$ aplica dos veces $\operatorname{dequant}$ para obtener los valores de los pesos originales en precisión _BF16_ para el _forward pass_ durante el _fine-tuning_.

- $ \operatorname{doubleDequant}(c_1^{\text{FP32}}, c_2^{\text{FP8}}, W^{\text{NF4}}) = \operatorname{dequant}\left(\operatorname{dequant}(c_1^{\text{FP32}}, c_2^{\text{FP8}}), W^{\text{NF4}} \right) = W^{\text{BF16}} $